In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)


CUDA available: True


In [12]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activation = activation

        self.fc_layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.fc_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size

        self.output_layer = nn.Linear(prev_size, output_size)

    def forward(self, x):
        x = x.view(-1, self.input_size)
        for fc_layer in self.fc_layers:
            x = self.activation(fc_layer(x))
        x = self.output_layer(x)
        return x

In [13]:
from sklearn.datasets import make_regression

# Generate a regression dataset with 100 samples, 5 input features, and 2 output targets
X, y = make_regression(n_samples=100, n_features=5, n_targets=2)


# Print the shape of the input and output arrays
print("Input shape:", X.shape)
print("Output shape:", y.shape)
# print(y[0])

Input shape: (100, 5)
Output shape: (100, 2)


In [17]:
from sklearn.model_selection import train_test_split
import numpy as np
import time

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into PyTorch tensors and move them to the GPU
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.FloatTensor(y_train).to(device)
y_test = torch.FloatTensor(y_test).to(device)

# Define the model architecture and move it to the GPU
input_size = X_train.shape[1]
hidden_sizes = [128, 64, 128, 128]
output_size = y_train.shape[1]
model = MLP(input_size, hidden_sizes, output_size, F.relu).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.MSELoss()

# Number of epochs
n_epochs = 1000

# Placeholder for losses
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

# Get the current time before the for loop
start_time = time.time()

for it in range(n_epochs):
    # zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    # Save losses
    train_losses[it] = loss.item()

    # Test loss
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    test_losses[it] = test_loss.item()

    if (it + 1) % 50 == 0:
        print(f'Epoch {it+1}/{n_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

# Get the current time after the for loop
end_time = time.time()

# Calculate the time difference
time_difference = end_time - start_time

# Print the time difference
print("Time difference:", time_difference)


Using GPU
Epoch 50/1000, Train Loss: 1772.3743, Test Loss: 2292.2039
Epoch 100/1000, Train Loss: 1151.1371, Test Loss: 1753.6592
Epoch 150/1000, Train Loss: 1126.7355, Test Loss: 1703.7614
Epoch 200/1000, Train Loss: 1093.1998, Test Loss: 1647.5127
Epoch 250/1000, Train Loss: 989.8803, Test Loss: 1368.6438
Epoch 300/1000, Train Loss: 751.1677, Test Loss: 617.3843
Epoch 350/1000, Train Loss: 630.6638, Test Loss: 437.7563
Epoch 400/1000, Train Loss: 569.3798, Test Loss: 525.4265
Epoch 450/1000, Train Loss: 281.0472, Test Loss: 202.0426
Epoch 500/1000, Train Loss: 147.5641, Test Loss: 155.4199
Epoch 550/1000, Train Loss: 68.1587, Test Loss: 120.6239
Epoch 600/1000, Train Loss: 25.5614, Test Loss: 99.4530
Epoch 650/1000, Train Loss: 15.1283, Test Loss: 81.1459
Epoch 700/1000, Train Loss: 34.6975, Test Loss: 122.8010
Epoch 750/1000, Train Loss: 8.4202, Test Loss: 67.6642
Epoch 800/1000, Train Loss: 6.6645, Test Loss: 63.3072
Epoch 850/1000, Train Loss: 5.5180, Test Loss: 60.4070
Epoch 900/1